# **Esercitazione 2 - Regressione Lineare**

## Boston Housing dataset

Questo dataset contiene informazioni raccolte dal U.S. Census Service riguardanti le abitazioni nell'area di Boston, Massachusetts. È stato ottenuto dall'archivio StatLib (http://lib.stat.cmu.edu/datasets/boston) ed è stato ampiamente utilizzato in letteratura per fare benchmark di algoritmi. 

Il dataset contiene informazioni su 506 case, divise in 14 variabili.

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import pandas as pd 
from sklearn.utils import shuffle
from pandas import read_csv

from sklearn.datasets import fetch_openml
import pandas as pd

# Scarica il Boston Housing Dataset da OpenML
boston = fetch_openml(name="Boston", version=1, as_frame=True)

# Estrai i dati (features) e il target (valore mediano delle abitazioni)
X = boston.data
y = boston.target

X, y = shuffle(X, y, random_state=0)
print(f"Features shape: {X.shape}, targets shape:  {y.shape}")

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


Features shape: (506, 13), targets shape:  (506,)


## `np.c_` in NumPy

L'oggetto `np.c_` in NumPy è una **scorciatoia** per concatenare array lungo il secondo asse (cioè, le colonne).

## Utilizzo
```python
np.c_[array1, array2, ...]


In [5]:
import numpy as np

# Generate two random 2x3 matrices
matrice1 = np.random.rand(2, 3)
matrice2 = np.random.rand(2, 3)

# Concatenate the matrices along columns
risultato = np.c_[matrice1, matrice2]

print("Matrice 1:",matrice1.shape)

print("\nMatrice 2:",matrice2.shape)

print("\nMatrice concatenata:",risultato.shape)

Matrice 1: (2, 3)

Matrice 2: (2, 3)

Matrice concatenata: (2, 6)


**Divisione del dataset**

Il primo passaggio è quello di dividere i dati in train set, validation set e test set. Utilizza il 60% dei dati per il training set, il 20% per il validation e il restante 20% per il test set. Considerato che il nostro dataset possiede 506 osservazioni mi aspetto che:

- Il **training set** avrà 303 osservazioni.
- Il **validation set** avrà 101 osservazioni.
- Il **test set** avrà 101 osservazioni.

In reatà il test set avrà 102 osservazioni per via delle approssimazioni.



In [36]:
# Divisione del dataset

train_porzione = 0.6  
val_porzione = 0.2  
test_porzione = 0.2

# svolgimento...

num_train=int(train_porzione*X.shape[0])
num_validation=int(val_porzione*X.shape[0])
X_train= X[:num_train]
y_train= y[:num_train]

X_validation= X[num_train:num_train + num_validation]
y_validation= y[num_train:num_train + num_validation]

X_test= X[num_train + num_validation:]
y_test= y [num_train + num_validation:]

print("Train shape", X_train.shape)
print("Validation shape", X_validation.shape)
print("Test shape", X_test.shape)

Train shape (303, 13)
Validation shape (101, 13)
Test shape (102, 13)


### **Esercizio 1: Costruisci una Pipeline di Regressione Lineare Standardizzata**

**Step 1:** Standardizza i dataset di addestramento, validazione e test. Usa `StandardScaler` di scikit-learn.  

**Step 2:** Aggiungi una feature costante (bias) ai dati concatenando una colonna di uno ad ogni dataset.  

**Step 3:** Implementa la soluzione in forma chiusa per l'addestramento di un modello di regressione lineare. 
 
**Step 4:** Valuta il modello calcolando il Mean Absolute Error (MAE) sui dataset di addestramento, validazione e test.


### **Guida**

1. **StandardScaler**:
   - Utilizza `StandardScaler` da `sklearn.preprocessing` per standardizzare i dati.
   - Il metodo `fit_transform` calcola la media e la varianza dei dati di addestramento e li scala di conseguenza.
   - Utilizza `transform` per standardizzare i dati di validazione e test utilizzando gli stessi parametri. Utilizziamo il metodo `transform` perchè non calcola i parametri di scaling (media e std). In questo modo ci assicuriamo che i dati di training e quelli di validation e test vengano scalati in modo uguale. Se usassimo `fit_transform` avremmo degli scaling diversi.

2. **Aggiunta di una Caratteristica Costante**:
   - Utilizza `np.c_` per concatenare una colonna di uno alle matrici delle caratteristiche. Questo è importante per includere il termine di intercetta nella regressione lineare.

3. **Soluzione in Forma Chiusa per la Regressione Lineare**:
   - La soluzione in forma chiusa è:

     $$\theta = (X^T X)^{-1} X^T y$$

   - Per calcolare la trasposta di una matrice possiamo utilizzare l' attributo `.T` di cui ogni array è dotato.

   - Utilizza `np.linalg.inv` di NumPy per l'inversione della matrice e l'operatore `@` per la moltiplicazione matriciale.
  
   - Puoi utilizzare l'operatore @ per eseguire l'operazione np.dot (`A @ B` è equivalente a `np.dot(A, B)`).

4. **Mean Absolute Error (MAE)**:
   - L'MAE si calcola come:

     $$\text{MAE} = \frac{1}{n} \sum_{i=1}^n |y_i - \hat{y}_i|$$

   - Utilizza `np.mean` e `np.abs` per calcolarlo.


In [37]:
# Step 1 - Normalizzazione dei dati. Dobbiamo normalizzare le features 
# sia del training set, validation set e test set.

# Utilizziamo il metodo .fit_transform() dello scaler per normalizzare le feature di training.

# Per normalizzare le feature di validation e test utilizziamo il metodo .transform()

from sklearn.preprocessing import StandardScaler

# svolgimento...

scaler= StandardScaler()
X_train= scaler.fit_transform(X_train)
X_validation= scaler.transform(X_validation)
X_test= scaler.transform(X_test)



In [38]:
# Step 2 - Aggiunta di una feature costante

# creiamo un vettore di 1 da aggiungere come feature costante. 
# ATTENZIONE: questo vettore deve avere le stesse righe del set a cui viene aggiunto. 
# Uno uguale per tutti non va bene

# svolgimento...

vettore_train= np.ones(X_train.shape[0])
vettore_val= np.ones(X_validation.shape[0])
vettore_test= np.ones(X_test.shape[0])

X_train_new= np.c_[vettore_train, X_train]
X_val_new= np.c_[vettore_val, X_validation]
X_test_new= np.c_[vettore_test, X_test]

print(X_train_new.shape)
print(X_val_new.shape)
print(X_test_new.shape)


(303, 14)
(101, 14)
(102, 14)


In [39]:
# Step 3 - Applichiamo la formula matematica della regressione lineare

# ATTENZIONE: stiamo per effettuare operazioni tra matrici e vettori, 
# non si tratta di una semplice formula matematica, stiamo attenti a quali operatori utilizzare e quanto

# svolgimento...

parametri = np.linalg.inv(X_train_new.T @ X_train_new) @ X_train_new.T @ y_train

print(parametri.shape)
print(parametri)

(14,)
[22.16138614 -0.50926449  1.04570167  0.08640982  0.43014273 -1.8975044
  2.62964478 -0.02436637 -2.95441032  2.92892785 -2.41571656 -1.74144698
  1.12132994 -3.56543566]


In [40]:
# Step 4 - Calcolo MAE

# Calcoliamo l'errore medio assoluto (MAE) per il training set, validation set e test set.
# Utlizziamo la formula specificata nella guida.

# svolgimento...

y_train_prediction = X_train_new @ parametri
y_val_prediction = X_val_new @ parametri
y_test_prediction = X_test_new @ parametri

print(y_train_prediction.shape)
print(y_val_prediction.shape)
print(y_test_prediction.shape)


mae_train=np.mean(np.abs(y_train - y_train_prediction))
mae_val=np.mean(np.abs(y_validation - y_val_prediction))
mae_test=np.mean(np.abs(y_test - y_test_prediction))

print("Mae trainset",mae_train)
print("Mae validationset", mae_val)
print("Mae testset",mae_test)

(303,)
(101,)
(102,)
Mae trainset 3.2496837441036983
Mae validationset 3.5977784744283965
Mae testset 3.0197366706166866


### **Esercizio: Costruisci una pipeline di Regressione Lineare Standardizzata utilizzando `scikit-learn`** 

**Step 1 & 2:** Step 1 e 2 sono uguali a quanto fatto prima.

**Step 3:** Utilizza `LinearRegression()` di scikit-learn per addestrare un modello di regressione lineare.  

**Step 4:** Valuta il modello calcolando il Mean Absolute Error (MAE) sui dataset di addestramento, validazione e test, utilizzando `mean_absolute_error()` da `sklearn.metrics`.


## `LinearRegression` da Scikit-Learn

La classe `LinearRegression` in Scikit-Learn viene utilizzata per eseguire la **regressione lineare**, adattando un modello lineare al dataset.

## **Sintassi**
```python
from sklearn.linear_model import LinearRegression

model = LinearRegression()
# Dati di esempio
X = np.array([[1, 1], [1, 2], [2, 2], [2, 3]])
y = np.array([10, 15, 20, 25])

# Adatta il modello ai dati
model.fit(X, y)

# Predici nuovi valori
X_new = np.array([[3, 5], [5, 9]])
predictions = model.predict(X_new)


## `mean_absolute_error` da Scikit-Learn

La funzione `mean_absolute_error` calcola l'**errore assoluto medio** (MAE) tra i valori target reali e quelli predetti.

## **Sintassi**
```python
sklearn.metrics.mean_absolute_error(y_true, y_pred)


### **Guida**

1. **Istanziare e allenare un modello di regressione lineare**:
    
    - Istanziamo una classe `LinearRegression` per creare il modello.
    - Utilizziamo il metodo `.fit()` per allenare il modello con i dati di training.

2. **Effettuare predizioni con il modello**:

    - Utiliziamo il metodo `.predict()` del modello per effettuare le predizioni. Effettuiamo le predizioni per tutti i set che abbiamo (train, validation e test).

3. **Calcolo della MAE**: 

    - Calcolare MAE su tutti i set utilizzando la funzione `mean_abslute_error`


In [41]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [42]:
# Step 1 - Istanziare e allenare il modello di regressione lineare.

# svolgimento...
model= LinearRegression()

model.fit(X_train_new,y_train)

LinearRegression()

In [43]:
# Step 2 - Effettuare predizioni

# svolgimento...

train_prediction=model.predict(X_train_new)

validation_prediction=model.predict(X_val_new)

test_prediction=model.predict(X_test_new)

print(train_prediction.shape)
print(validation_prediction.shape)
print(test_prediction.shape)

(303,)
(101,)
(102,)


In [44]:
# Step 3 - Calcolo MAE

# svolgimento...



mae_train= mean_absolute_error(y_train, train_prediction)
mae_val= mean_absolute_error(y_validation, validation_prediction)
mae_test= mean_absolute_error(y_test, test_prediction)

print("mae train",mae_train)
print("mae val", mae_val)
print("mae_test",mae_test)

mae train 3.2496837441036983
mae val 3.5977784744283983
mae_test 3.019736670616686


### **Esercizio: Crea una funzione che esegua una pipeline di Regressione Lineare**

La funzione deve richiedere un parametro `hyperparams` per gestire i diversi casi. 

`hyperparams` deve essere un dizionario contenente diverse chiavi, in base al valore di queste chiavi devono essere eseguiti (oppure no) diversi pezzi di codice. 

In questo esercizio la chiave da utilizzare sarà `hyperparams['data_standardize']`. Se il valore di questa chiave sarà **True** allora eseguire la standardizzazione con `scikit-learn`, se invece è **False** non verrà eseguita alcuna standardizzazione.

**Step 1:** Controllare se eseguire o no la standardizzazione.

* **Step 1.1:** Scrivere il codice per eseguire la standardizzazione.

**Step 2:** Utilizza `np.c_` per concatenare una colonna di uno alle matrici delle caratteristiche.

**Step 3:** Applichiamo la formula matematica della regressione lineare.

**Step 4:** Calcolo MAE utilizzando la formula (NON con `scikit-learn`).

La funzione deve ritornare i valori della MAE.

Dopo aver testato i risultati con `hyperparams['data_standardize']` = **True**, provare anche i risultati ottenuti se `hyperparams['data_standardize']` = **False**.

In [45]:
# svolgimento...

def pipeline(X_train, y_train, X_val, y_val, hyperparams):

    X_train = np.array(X_train, dtype=float)
    y_train = np.array(y_train, dtype=float)
    X_val = np.array(X_val, dtype=float)
    y_val = np.array(y_val, dtype=float)
    
    # Step 1 - Controllo se è richiesta la standardizzazione dei dati
    if hyperparams['data_standardize']:
        
        # Step 1.1 - Scrivere il codice per standardizzare i dati 
            scaler = StandardScaler()
            X_train = scaler.fit_transform(X_train)
            X_val = scaler.transform(X_val)
    
    # Step 2 - Concatenare una colonna di uno alla matrice delle features
            X_train = np.c_[np.ones(X_train.shape[0]), X_train]
            X_val = np.c_[np.ones(X_val.shape[0]), X_val]
    # Step 3 - Applicare formula della regressione lineare e calcolare predizioni
            parametri = np.linalg.inv(X_train.T @ X_train) @ X_train.T @ y_train
            val_prediction = X_val @ parametri
            train_prediction = X_train @ parametri
    # Step 4 - Calcolare MAE 
            mae_train = np.mean(np.abs(train_prediction - y_train))
            mae_val = np.mean(np.abs(val_prediction - y_val))
    
    return mae_train, mae_val


In [46]:
hyperparams = {'data_standardize': True}

train_fraction = 0.8
validation_fraction = 0.2

num_train = int(train_fraction * X.shape[0])

X_train = X[:num_train]
y_train = y[:num_train]

X_validation = X[num_train:]
y_validation = y[num_train:]

# Chiamare la funzione pipeline e stampare i risultati della MAE

# svolgimento...
mae_train, mae_val = pipeline(X_train, y_train, X_validation, y_validation, hyperparams)
print("Mae train", mae_train)
print("Mae validation", mae_val)


Mae train 3.3692123106941234
Mae validation 3.023751047336023


### **Esercizio: Implementare alla funzione `pipeline` la possibilità di usare PCA**

Modifichiamo la funzione `pipeline` in modo da gestire anche la possibilità di effettuare la PCA. Dunque aggiungiamo al dizionario `hyperparams` la chiave `use_pca`. 

Se `hyperparams['use_pca']` = **True** verrà eseguita la PCA. 

Se `hyperparams['use_pca']` = **False** non verrà eseguita la PCA.

La gestione della standardizzazione deve essere mantenuta come prima.

In [48]:
# svolgimento...
from sklearn.decomposition import PCA

def pipeline(X_train, y_train, X_val, y_val, hyperparams):

    X_train = np.array(X_train, dtype=float)
    y_train = np.array(y_train, dtype=float)
    X_val = np.array(X_val, dtype=float)
    y_val = np.array(y_val, dtype=float)

    # Step 1 - Controllo se è richista la PCA
    if hyperparams['use_pca']:
        
        # Step 1.1 - Scrivere il codice per applicare PCA #per usarla devo usare i dati gia standardizzati
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_val = scaler.transform(X_val)
        
        n_components=2
        pca = PCA(n_components)
        X_train = pca.fit_transform(X_train)
        X_val = pca.transform(X_val)
    # Step 2 - Controllo se è richiesta la standardizzazione dei dati
    if hyperparams['data_standardize']:
        
        # Step 2.1 - Scrivere il codice per standardizzare i dati 
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_val = scaler.transform(X_val)

    # Step 3 - Concatenare una colonna di uno alla matrice delle features
        X_train = np.c_[np.ones(X_train.shape[0]), X_train]
        X_val = np.c_[np.ones(X_val.shape[0]), X_val]
    # Step 4 - Applicare formula della regressione lineare e calcolare predizioni
        parametri = np.linalg.inv(X_train.T @ X_train) @ X_train.T @ y_train
        train_prediction = X_train @ parametri
        val_prediction = X_val @ parametri
        
    # Step 5 - Calcolare MAE 
        mae_train = np.mean(np.abs(train_prediction - y_train))
        mae_val = np.mean(np.abs(val_prediction- y_val))

    return mae_train, mae_val



In [51]:
hyperparams = {'data_standardize': True, 'use_pca': True}
train_fraction = 0.8
validation_fraction = 0.2

num_train = int(train_fraction * X.shape[0])

X_train = X[:num_train]
y_train = y[:num_train]

X_validation = X[num_train:]
y_validation = y[num_train:]

# Chiamare la funzione pipeline e stampare i risultati della MAE al variare dell' utilizzo della PCA.

# svolgimento...

mae_train, mae_val= pipeline(X_train, y_train, X_validation, y_validation, hyperparams)
print("Mae train", mae_train)
print("Mae val", mae_val)

#ho fatto il test mettendo use_pca=false e ho visto che i valori combaciano con quanto fatto prima, giusto per verificare

Mae train 4.894102015491301
Mae val 4.843264298439217
